In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from cycler import cycler
import matplotlib.font_manager as fm

# Paths to the Computer Modern font files
font_paths = {
    'serif': 'assets/fonts/cmunrm.ttf',
    'italic': 'assets/fonts/cmunti.ttf',
    'bold': 'assets/fonts/cmunbx.ttf',
    'bold_italic': 'assets/fonts/cmunbi.ttf'
}

# Add the fonts to Matplotlib
for key, path in font_paths.items():
    fm.fontManager.addfont(path)

# Set the font properties globally
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['CMU Serif']
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Monochrome color cycle
monochrome = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':', '-.']))
rcParams['axes.prop_cycle'] = monochrome

# Function to format titles (only first letter capitalized)
def format_title(title):
    return title.capitalize()

# Function to add subfigure labels
def add_subfigure_label(ax, label):
    ax.text(-0.1, 1.1, f'({label})', transform=ax.transAxes, fontsize=14, fontweight='bold', va='top')

# ... rest of your code

# Helper function to create unique method names
def get_unique_method_name(method, params):
    if method == 'wv_ols_r2':
        return f"wv_ols_r2_{str(params.get('emphasize_diversity', False))[0]}"
    elif method == 'hard_voting':
        return f"hard_voting_{str(params.get('gaussian', False))[0]}"
    elif params:
        param_str = '_'.join(f"{k}_{v}" for k, v in sorted(params.items()))
        return f"{method}_{param_str}"
    return method

# Load the data
e1_results = pd.read_csv('final_experiments/experiment_e1_results.csv')
e2_results = pd.read_csv('final_experiments/experiment_e2_results.csv')
e2_summary = pd.read_csv('final_experiments/experiment_e2_summary.csv')

# Load configurations
with open('final_experiments/experiment_e2_configurations.json', 'r') as f:
    e2_configs = json.load(f)

# Create unique method names
e2_results['unique_method'] = e2_results.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)
e2_summary['unique_method'] = e2_summary.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)

# Calculate relative performance for each ensemble
relative_performances = []
for config in e2_configs:
    bl_type = config['base_learner_type']
    method = config['ensemble_method']
    rep = config['repetition']
    constituent_scores = [e1_results[(e1_results['base_learner'] == bl_type) & 
                                     (e1_results['params'] == str(params))]['ucr_score'].values[0]
                          for params in config['base_learner_params']]
    avg_constituent_score = np.mean(constituent_scores)
    unique_method = get_unique_method_name(method, config['method_params'])
    ensemble_data = e2_results[(e2_results['base_learner_type'] == bl_type) & 
                               (e2_results['unique_method'] == unique_method) & 
                               (e2_results['repetition'] == rep)]
    ensemble_score = ensemble_data['ucr_score'].values[0]
    computational_time = ensemble_data['computational_time'].values[0]
    relative_score = ensemble_score / avg_constituent_score
    relative_performances.append({
        'base_learner_type': bl_type,
        'unique_method': unique_method,
        'repetition': rep,
        'relative_score': relative_score,
        'ensemble_score': ensemble_score,
        'avg_constituent_score': avg_constituent_score,
        'computational_time': computational_time
    })

relative_df = pd.DataFrame(relative_performances)



# 1. Comparative Box Plots
fig, axs = plt.subplots(2, 2, figsize=(20, 15))
base_learners = e2_results['base_learner_type'].unique()
for i, bl in enumerate(base_learners):
    ax = axs[i // 2, i % 2]
    data = []
    labels = []
    for method in relative_df['unique_method'].unique():
        data.append(relative_df[(relative_df['base_learner_type'] == bl) & 
                                (relative_df['unique_method'] == method)]['relative_score'])
        labels.append(method)
    ax.boxplot(data, labels=labels)
    ax.axhline(y=1, color='k', linestyle='--')
    ax.set_title(format_title(f'{bl} - ensemble methods'))
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_ylabel('Relative score')
    add_subfigure_label(ax, chr(97 + i))  # 'a', 'b', 'c', 'd'

fig.suptitle(format_title('Comparative performance of ensemble methods'), fontsize=16)
plt.tight_layout()
plt.savefig('final_visualisations/experiment_2/comparative_box_plots.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.close()

# 2. Performance Improvement Heatmap
improvement_data = relative_df.groupby(['base_learner_type', 'unique_method'])['relative_score'].mean().reset_index()
improvement_pivot = improvement_data.pivot(index='base_learner_type', columns='unique_method', values='relative_score')
improvement_pivot = (improvement_pivot - 1) * 100  # Convert to percentage improvement

fig, ax = plt.subplots(figsize=(20, 12))
sns.heatmap(improvement_pivot, annot=True, fmt='.2f', cmap='Greys', ax=ax)

ax.set_title(format_title('Average performance improvement of ensemble methods (%)'))
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
add_subfigure_label(ax, 'a')

plt.tight_layout()
plt.savefig('final_visualisations/experiment_2/performance_improvement_heatmap.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.close()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/seaborn/utils.py:61: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  fig.canvas.draw()
/tmp/ipykernel_48492/21101144.py:135: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.savefig('final_visualisations/experiment_2/performance_improvement_heatmap.pdf', format='pdf', dpi=300, bbox_inches='tight')
/tmp/ipykernel_48492/21101144.py:135: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.savefig('final_visualisations/experiment_2/performance_improvement_heatmap.pdf', format='pdf', dpi=300, bbox_inches='tight')


In [1]:
# import cudf.pandas
# cudf.pandas.install()

# import pyscamp

# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import random
# import os
# import json

# from lof import LocalOutlierFactor
# from matrix_profile import MatrixProfile
# from isolation_forest import IsolationForest
# from kmeans import KMeans
# from ensemble_detector import EnsembleDetector
# from dataloader import DataLoader
# from benchmarker import benchmark

# # Constants
# UCR_PATH = 'ucrdata'
# CACHED_SCORES_DIR = 'final_scores'
# RESULTS_DIR = 'final_results'
# ENSEMBLE_RESULTS_DIR = 'final_ensembles/results'
# ENSEMBLE_SCORES_DIR = 'final_ensembles/scores'
# N_REPETITIONS = 30
# ENSEMBLE_SIZE =16

# # Base learner configurations
# BASE_LEARNERS = {
#     'LOF': {
#         'class': LocalOutlierFactor,
#         'params': [
#             {'windowSize': ws, 'neighbors': n, 'gpu': True}
#             for ws in [25, 50, 100, 150, 200]
#             for n in [10, 20, 50, 100]
#         ]
#     },
#     'IF': {
#         'class': IsolationForest,
#         'params': [
#             {'windowSize': ws}
#             for ws in [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700]
#         ]
#     },
#     'KMeans': {
#         'class': KMeans,
#         'params': [
#             {'windowSize': ws, 'n_clusters': nc}
#             for ws in [50, 100, 200, 500]
#             for nc in [10, 20, 50, 100, 200]
#         ]
#     },
#     'MP': {
#         'class': MatrixProfile,
#         'params': [
#             {'windowSize': ws}
#             for ws in [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700]
#         ]
#     }
# }

# # Ensemble methods
# ENSEMBLE_METHODS = [
#     ('simple_average', {}),
#     ('maximum_score', {}),
#     ('wv_ols', {}),
#     ('wv_ols_r2', {'emphasize_diversity': False}),
#     # ('wv_ols_r2', {'emphasize_diversity': True}),
#     ('hard_voting', {'spread_window': 100, 'gaussian': False}),
#     ('hard_voting', {'spread_window': 100, 'gaussian': True})

# ]

# def create_homogeneous_ensemble(base_learner_type, n=ENSEMBLE_SIZE):
#     base_learner_class = BASE_LEARNERS[base_learner_type]['class']
#     params_list = BASE_LEARNERS[base_learner_type]['params']
#     selected_params = random.sample(params_list, n)
#     return [base_learner_class(**params) for params in selected_params], selected_params

# def run_experiment():
#     results = []
#     configurations = []
    
#     for base_learner_type in BASE_LEARNERS.keys():
#         for rep in tqdm(range(N_REPETITIONS), desc=f"Running {base_learner_type} ensembles"):
#             base_learners, selected_params = create_homogeneous_ensemble(base_learner_type)
            
#             for method, method_params in ENSEMBLE_METHODS:
#                 ensemble = EnsembleDetector(
#                     base_learners=base_learners,
#                     method=method,
#                     method_params=method_params,
#                     scores_dir=CACHED_SCORES_DIR
#                 )
                
#                 ensemble_name = ensemble.toString()
#                 save_results_file = os.path.join(ENSEMBLE_RESULTS_DIR, f"{ensemble_name}_rep{rep}.csv")
#                 save_scores_dir = os.path.join(ENSEMBLE_SCORES_DIR, f"{ensemble_name}_rep{rep}")
                
#                 benchmark(ensemble, UCR_PATH, save_results_file, save_scores_dir)
                
#                 benchmark_results = pd.read_csv(save_results_file, nrows=1)
                
#                 result = {
#                     'base_learner_type': base_learner_type,
#                     'ensemble_method': method,
#                     'method_params': json.dumps(method_params),  # Convert dict to string for storage in DataFrame
#                     'repetition': rep,
#                     'ucr_score': benchmark_results['accuracy'].values[0],
#                     'computational_time': benchmark_results['total_time'].values[0]
#                 }
#                 results.append(result)
                
#                 configuration = {
#                     'base_learner_type': base_learner_type,
#                     'ensemble_method': method,
#                     'method_params': method_params,
#                     'repetition': rep,
#                     'base_learner_params': selected_params
#                 }
#                 configurations.append(configuration)
    
#     return pd.DataFrame(results), configurations

# def analyze_results(results_df):
#     summary = results_df.groupby(['base_learner_type', 'ensemble_method', 'method_params']).agg({
#         'ucr_score': ['mean', 'std', 'max'],
#         'computational_time': ['mean', 'std']
#     }).reset_index()
    
#     summary.columns = ['base_learner_type', 'ensemble_method', 'method_params', 'mean_score', 'std_score', 'max_score', 'mean_time', 'std_time']
#     summary['cv_score'] = summary['std_score'] / summary['mean_score']
    
#     return summary

# # # Run the experiment
# # if not os.path.exists(ENSEMBLE_RESULTS_DIR):
# #     os.makedirs(ENSEMBLE_RESULTS_DIR)
# # if not os.path.exists(ENSEMBLE_SCORES_DIR):
# #     os.makedirs(ENSEMBLE_SCORES_DIR)

# # results_df, configurations = run_experiment()
# # results_df.to_csv('final_experiments/experiment_e2_results.csv', index=False)

# # # Save configurations
# # with open('final_experiments/experiment_e2_configurations.json', 'w') as f:
# #     json.dump(configurations, f, indent=2)

# results_df = pd.read_csv('final_experiments/experiment_e2_results.csv')

# # Analyze the results
# summary = analyze_results(results_df)
# summary.to_csv('final_experiments/experiment_e2_summary.csv', index=False)

# print("Experiment E2 completed. Results saved to 'final_experiments/experiment_e2_results.csv', 'final_experiments/experiment_e2_configurations.json', and 'final_experiments/experiment_e2_summary.csv'.")





Experiment E2 completed. Results saved to 'final_experiments/experiment_e2_results.csv', 'final_experiments/experiment_e2_configurations.json', and 'final_experiments/experiment_e2_summary.csv'.


In [2]:
# # Run the experiment
# if not os.path.exists(ENSEMBLE_RESULTS_DIR):
#     os.makedirs(ENSEMBLE_RESULTS_DIR)
# if not os.path.exists(ENSEMBLE_SCORES_DIR):
#     os.makedirs(ENSEMBLE_SCORES_DIR)

# results_df, configurations = run_experiment()
# results_df.to_csv('experiment_e2_results.csv', index=False)

# # Save configurations
# with open('experiment_e2_configurations.json', 'w') as f:
#     json.dump(configurations, f, indent=2)

# Analyze the results
summary = analyze_results(results_df)
summary.to_csv('experiment_e2_summary.csv', index=False)

print("Experiment E2 completed. Results saved to 'experiment_e2_results.csv', 'experiment_e2_configurations.json', and 'experiment_e2_summary.csv'.")
summary = pd.read_csv('experiments/experiment_e2_summary.csv')

# Compare with individual base learners from E1
e1_results = pd.read_csv('experiments/experiment_e1_results.csv')
e1_summary = e1_results.groupby('base_learner').agg({
    'ucr_score': ['mean', 'std', 'max'],
    'computational_time': ['mean', 'std']
}).reset_index()
e1_summary.columns = ['base_learner', 'mean_score', 'std_score', 'max_score', 'mean_time', 'std_time']
e1_summary['cv_score'] = e1_summary['std_score'] / e1_summary['mean_score']

# print("\nComparison with individual base learners from E1:")
# print(e1_summary)
# print("\nHomogeneous ensemble summary:")
# print(summary)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from matplotlib.colors import LinearSegmentedColormap

# Updated function to create unique method names
def get_unique_method_name(method, params):
    if method == 'wv_ols_r2':
        return f"wv_ols_r2_{str(params.get('emphasize_diversity', False))[0]}"
    elif method == 'hard_voting':
        return f"hard_voting_{str(params.get('gaussian', False))[0]}"
    elif params:
        param_str = '_'.join(f"{k}_{v}" for k, v in sorted(params.items()))
        return f"{method}_{param_str}"
    return method

# Load the data
e1_results = pd.read_csv('experiments/experiment_e1_results.csv')
e2_results = pd.read_csv('experiments/experiment_e2_results.csv')
e2_summary = pd.read_csv('experiments/experiment_e2_summary.csv')

# Create unique method names
e2_results['unique_method'] = e2_results.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)
e2_summary['unique_method'] = e2_summary.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)

# Load configurations
with open('experiments/experiment_e2_configurations.json', 'r') as f:
    e2_configs = json.load(f)

# 1. Comparative Box Plots
plt.figure(figsize=(20, 15))
base_learners = e2_results['base_learner_type'].unique()
for i, bl in enumerate(base_learners, 1):
    plt.subplot(2, 2, i)
    data = [e1_results[e1_results['base_learner'] == bl]['ucr_score']]
    labels = ['Individual']
    for method in e2_results['unique_method'].unique():
        data.append(e2_results[(e2_results['base_learner_type'] == bl) & 
                               (e2_results['unique_method'] == method)]['ucr_score'])
        labels.append(method)
    plt.boxplot(data, labels=labels)
    plt.title(f'{bl} - Individual vs Ensemble Methods')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('UCR Score')
plt.tight_layout()
plt.savefig('visualisations/experiment_2/comparative_box_plots.png')
plt.close()

# 2. Performance Improvement Heatmap
improvement_data = []
for bl in base_learners:
    bl_avg = e1_results[e1_results['base_learner'] == bl]['ucr_score'].mean()
    for method in e2_summary['unique_method'].unique():
        ensemble_avg = e2_summary[(e2_summary['base_learner_type'] == bl) & 
                                  (e2_summary['unique_method'] == method)]['mean_score'].values[0]
        improvement = (ensemble_avg - bl_avg) / bl_avg * 100
        improvement_data.append([bl, method, improvement])

improvement_df = pd.DataFrame(improvement_data, columns=['Base Learner', 'Ensemble Method', 'Improvement'])
improvement_pivot = improvement_df.pivot(index='Base Learner', columns='Ensemble Method', values='Improvement')

plt.figure(figsize=(20, 12))
colors = ['#FF0000', '#FFFFFF', '#00FF00']  # Red, White, Green
n_bins = 100  # Number of bins for color gradation
cmap = LinearSegmentedColormap.from_list('custom', colors, N=n_bins)

vmin = improvement_pivot.min().min()
vmax = improvement_pivot.max().max()
vcenter = 0

sns.heatmap(improvement_pivot, annot=True, fmt='.2f', cmap=cmap, 
            vmin=vmin, vmax=vmax, center=vcenter)

plt.title('Performance Improvement of Ensemble Methods (%)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('visualisations/experiment_2/performance_improvement_heatmap.png')
plt.close()

# 3. Reliability Improvement Bar Chart
plt.figure(figsize=(20, 12))
base_learners = e1_results['base_learner'].unique()
ensemble_methods = e2_summary['unique_method'].unique()

x = np.arange(len(base_learners))
width = 0.8 / (len(ensemble_methods) + 1)  # +1 for the individual learner bar

e1_cv = e1_results.groupby('base_learner')['ucr_score'].apply(lambda x: x.std() / x.mean())
plt.bar(x, e1_cv, width, label='Individual')

for i, method in enumerate(ensemble_methods, 1):
    cv_scores = []
    for bl in base_learners:
        cv = e2_summary[(e2_summary['base_learner_type'] == bl) & 
                        (e2_summary['unique_method'] == method)]['cv_score'].values
        cv_scores.append(cv[0] if len(cv) > 0 else np.nan)
    plt.bar(x + width * i, cv_scores, width, label=method)

plt.xlabel('Base Learner Type')
plt.ylabel('Coefficient of Variation of UCR Scores')
plt.title('Reliability Comparison: Individual vs Ensemble Methods')
plt.xticks(x + width * (len(ensemble_methods) / 2), base_learners)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.tight_layout()
plt.savefig('visualisations/experiment_2/reliability_improvement_bar_chart.png')
plt.close()

# 4. Computational Time vs. Performance Scatter Plot
plt.figure(figsize=(15, 10))
for bl in base_learners:
    e1_data = e1_results[e1_results['base_learner'] == bl]
    plt.scatter(e1_data['computational_time'], e1_data['ucr_score'], 
                label=f'{bl} (Individual)', alpha=0.7, marker='o')
    
    e2_data = e2_summary[e2_summary['base_learner_type'] == bl]
    plt.scatter(e2_data['mean_time'], e2_data['mean_score'], 
                label=f'{bl} (Ensemble)', alpha=0.7, marker='^')

plt.xlabel('Computational Time')
plt.ylabel('UCR Score')
plt.title('Performance vs Computational Time: Individual vs Ensemble Methods')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig('visualisations/experiment_2/performance_vs_time_scatter.png')
plt.close()

# 5. Performance Relative to Constituent Learners
relative_performance = []
for config in e2_configs:
    bl_type = config['base_learner_type']
    method = config['ensemble_method']
    rep = config['repetition']
    constituent_scores = [e1_results[(e1_results['base_learner'] == bl_type) & 
                                     (e1_results['params'] == str(params))]['ucr_score'].values[0]
                          for params in config['base_learner_params']]
    avg_constituent_score = np.mean(constituent_scores)
    unique_method = get_unique_method_name(method, config['method_params'])
    ensemble_score = e2_results[(e2_results['base_learner_type'] == bl_type) & 
                                (e2_results['unique_method'] == unique_method) & 
                                (e2_results['repetition'] == rep)]['ucr_score'].values[0]
    relative_score = ensemble_score / avg_constituent_score
    relative_performance.append([bl_type, unique_method, relative_score])

relative_df = pd.DataFrame(relative_performance, columns=['Base Learner', 'Ensemble Method', 'Relative Score'])

plt.figure(figsize=(20, 12))
sns.violinplot(x='Ensemble Method', y='Relative Score', hue='Base Learner', 
               data=relative_df, split=True)
plt.axhline(y=1, color='r', linestyle='--')
plt.title('Ensemble Performance Relative to Constituent Learners')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('visualisations/experiment_2/relative_performance_violin.png')
plt.close()

print("All visualizations have been created and saved in the 'visualisations/experiment_2' directory.")

All visualizations have been created and saved in the 'visualisations/experiment_2' directory.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from matplotlib.colors import LinearSegmentedColormap

# Helper function to create unique method names
def get_unique_method_name(method, params):
    if method == 'wv_ols_r2':
        return f"wv_ols_r2_{str(params.get('emphasize_diversity', False))[0]}"
    elif method == 'hard_voting':
        return f"hard_voting_{str(params.get('gaussian', False))[0]}"
    elif params:
        param_str = '_'.join(f"{k}_{v}" for k, v in sorted(params.items()))
        return f"{method}_{param_str}"
    return method

# Load the data
e1_results = pd.read_csv('experiments/experiment_e1_results.csv')
e2_results = pd.read_csv('experiments/experiment_e2_results.csv')
e2_summary = pd.read_csv('experiments/experiment_e2_summary.csv')

# Load configurations
with open('experiments/experiment_e2_configurations.json', 'r') as f:
    e2_configs = json.load(f)

# Create unique method names
e2_results['unique_method'] = e2_results.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)
e2_summary['unique_method'] = e2_summary.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)

# Calculate relative performance for each ensemble
relative_performances = []
for config in e2_configs:
    bl_type = config['base_learner_type']
    method = config['ensemble_method']
    rep = config['repetition']
    constituent_scores = [e1_results[(e1_results['base_learner'] == bl_type) & 
                                     (e1_results['params'] == str(params))]['ucr_score'].values[0]
                          for params in config['base_learner_params']]
    avg_constituent_score = np.mean(constituent_scores)
    unique_method = get_unique_method_name(method, config['method_params'])
    ensemble_data = e2_results[(e2_results['base_learner_type'] == bl_type) & 
                               (e2_results['unique_method'] == unique_method) & 
                               (e2_results['repetition'] == rep)]
    ensemble_score = ensemble_data['ucr_score'].values[0]
    computational_time = ensemble_data['computational_time'].values[0]
    relative_score = ensemble_score / avg_constituent_score
    relative_performances.append({
        'base_learner_type': bl_type,
        'unique_method': unique_method,
        'repetition': rep,
        'relative_score': relative_score,
        'ensemble_score': ensemble_score,
        'avg_constituent_score': avg_constituent_score,
        'computational_time': computational_time
    })

relative_df = pd.DataFrame(relative_performances)


# 1. Comparative Box Plots
plt.figure(figsize=(20, 15))
base_learners = e2_results['base_learner_type'].unique()
for i, bl in enumerate(base_learners, 1):
    plt.subplot(2, 2, i)
    # data = [e1_results[e1_results['base_learner'] == bl]['ucr_score']]
    data = []
    labels = []
    for method in relative_df['unique_method'].unique():
        data.append(relative_df[(relative_df['base_learner_type'] == bl) & 
                                (relative_df['unique_method'] == method)]['relative_score'])
        labels.append(method)
    plt.boxplot(data, labels=labels)
    plt.axhline(y=1, color='r', linestyle='--')
    plt.title(f'{bl} - Ensemble Methods')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Relative Score')
plt.tight_layout()
plt.savefig('visualisations/experiment_2b/comparative_box_plots.png')
plt.close()

# 2. Performance Improvement Heatmap
improvement_data = relative_df.groupby(['base_learner_type', 'unique_method'])['relative_score'].mean().reset_index()
improvement_pivot = improvement_data.pivot(index='base_learner_type', columns='unique_method', values='relative_score')
improvement_pivot = (improvement_pivot - 1) * 100  # Convert to percentage improvement

plt.figure(figsize=(20, 12))
colors = ['#FF0000', '#FFFFFF', '#00FF00']  # Red, White, Green
cmap = LinearSegmentedColormap.from_list('custom', colors, N=100)

vmin = improvement_pivot.min().min()
vmax = improvement_pivot.max().max()
vcenter = 0

sns.heatmap(improvement_pivot, annot=True, fmt='.2f', cmap=cmap, 
            vmin=vmin, vmax=vmax, center=vcenter)

plt.title('Average Performance Improvement of Ensemble Methods (%)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('visualisations/experiment_2b/performance_improvement_heatmap.png')
plt.close()

# 3. Reliability Improvement Bar Chart
plt.figure(figsize=(20, 12))
base_learners = e1_results['base_learner'].unique()
ensemble_methods = relative_df['unique_method'].unique()

x = np.arange(len(base_learners))
width = 0.8 / (len(ensemble_methods) + 1)  # +1 for the individual learner bar

e1_cv = e1_results.groupby('base_learner')['ucr_score'].apply(lambda x: x.std() / x.mean())
plt.bar(x, e1_cv, width, label='Individual')

for i, method in enumerate(ensemble_methods, 1):
    cv_scores = []
    for bl in base_learners:
        e1_cv = e1_results.groupby('base_learner')['ucr_score'].apply(lambda x: x.std() / x.mean())

        cv = relative_df[(relative_df['base_learner_type'] == bl) & 
                        (relative_df['unique_method'] == method)]['ensemble_score'].std() / \
            relative_df[(relative_df['base_learner_type'] == bl) & 
                        (relative_df['unique_method'] == method)]['ensemble_score'].mean()
        cv_scores.append(cv)
    plt.bar(x + width * i, cv_scores, width, label=method)

plt.xlabel('Base Learner Type')
plt.ylabel('Coefficient of Variation')
plt.title('Reliability Comparison: Individual vs Ensemble Methods')
plt.xticks(x + width * (len(ensemble_methods) / 2), base_learners)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.tight_layout()
plt.savefig('visualisations/experiment_2b/reliability_improvement_bar_chart.png')
plt.close()

# 4. Computational Time vs. Performance Scatter Plot
plt.figure(figsize=(15, 10))
for bl in base_learners:
    e1_data = e1_results[e1_results['base_learner'] == bl]
    plt.scatter(e1_data['computational_time'], e1_data['ucr_score'], 
                label=f'{bl} (Individual)', alpha=0.7, marker='o')
    
    e2_data = relative_df[relative_df['base_learner_type'] == bl]
    plt.scatter(e2_data['computational_time'], e2_data['relative_score'], 
                label=f'{bl} (Ensemble)', alpha=0.7, marker='^')

plt.xlabel('Computational Time')
plt.ylabel('UCR Score / Relative Score')
plt.title('Performance vs Computational Time: Individual vs Ensemble Methods')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig('visualisations/experiment_2b/performance_vs_time_scatter.png')
plt.close()

# 5. Performance Relative to Constituent Learners (Violin Plot)
plt.figure(figsize=(20, 12))
sns.violinplot(x='unique_method', y='relative_score', hue='base_learner_type', 
               data=relative_df, split=True)
plt.axhline(y=1, color='r', linestyle='--')
plt.title('Ensemble Performance Relative to Constituent Learners')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('visualisations/experiment_2b/relative_performance_violin.png')
plt.close()

print("All visualisations have been created and saved in the 'visualisations/experiment_2b' directory.")

All visualisations have been created and saved in the 'visualisations/experiment_2b' directory.


In [32]:
# relative_df[relative_df['unique_method']=='simple_average'].groupby('base_learner_type')['relative_score'].mean()
relative_df

,base_learner_type,unique_method,repetition,relative_score,ensemble_score,avg_constituent_score
0,LOF,simple_average,0,1.030303,0.544,0.5280
1,LOF,maximum_score,0,1.037879,0.548,0.5280
2,LOF,wv_ols,0,1.136364,0.600,0.5280
3,LOF,wv_ols_r2_F,0,1.098485,0.580,0.5280
4,LOF,wv_ols_r2_T,0,0.916667,0.484,0.5280
...,...,...,...,...,...,...
835,MP,wv_ols,29,1.253264,0.384,0.3064
836,MP,wv_ols_r2_F,29,0.535248,0.164,0.3064
837,MP,wv_ols_r2_T,29,1.201044,0.368,0.3064
838,MP,hard_voting_F,29,0.900783,0.276,0.3064


In [16]:
e1_results.head()

,params,ucr_score,computational_time,base_learner
0,"{'windowSize': 25, 'neighbors': 10, 'gpu': True}",0.572,593.591890,LOF
1,"{'windowSize': 25, 'neighbors': 20, 'gpu': True}",0.572,595.851824,LOF
2,"{'windowSize': 25, 'neighbors': 50, 'gpu': True}",0.576,604.703061,LOF
3,"{'windowSize': 25, 'neighbors': 100, 'gpu': True}",0.520,616.449885,LOF
4,"{'windowSize': 50, 'neighbors': 10, 'gpu': True}",0.572,1144.690072,LOF


In [17]:
e2_results.head(10)

,base_learner_type,ensemble_method,method_params,repetition,ucr_score,computational_time,unique_method
0,LOF,simple_average,{},0,0.544,4431.247839,simple_average
1,LOF,maximum_score,{},0,0.548,4431.594598,maximum_score
2,LOF,wv_ols,{},0,0.600,4443.556513,wv_ols
3,LOF,wv_ols_r2,"{""emphasize_diversity"": false}",0,0.580,4445.909986,wv_ols_r2_F
4,LOF,wv_ols_r2,"{""emphasize_diversity"": true}",0,0.484,4446.019717,wv_ols_r2_T
5,LOF,hard_voting,"{""spread_window"": 100, ""gaussian"": false}",0,0.560,4431.124438,hard_voting_F
6,LOF,hard_voting,"{""spread_window"": 100, ""gaussian"": true}",0,0.568,4431.433818,hard_voting_T
7,LOF,simple_average,{},1,0.572,3884.860332,simple_average
8,LOF,maximum_score,{},1,0.556,3885.207293,maximum_score
9,LOF,wv_ols,{},1,0.628,3896.149304,wv_ols


In [18]:
e2_summary

,base_learner_type,ensemble_method,method_params,mean_score,std_score,max_score,mean_time,std_time,cv_score,unique_method
0,IF,hard_voting,"{""spread_window"": 100, ""gaussian"": false}",0.218667,0.013425,0.236,200.911367,28.740103,0.061395,hard_voting_F
1,IF,hard_voting,"{""spread_window"": 100, ""gaussian"": true}",0.213867,0.014354,0.244,201.203889,28.728715,0.067119,hard_voting_T
2,IF,maximum_score,{},0.203733,0.011212,0.228,201.451223,28.734086,0.055035,maximum_score
3,IF,simple_average,{},0.216267,0.010657,0.232,201.085239,28.736838,0.049279,simple_average
4,IF,wv_ols,{},0.193467,0.017655,0.228,214.064810,28.371239,0.091257,wv_ols
5,IF,wv_ols_r2,"{""emphasize_diversity"": false}",0.194133,0.017663,0.228,215.275391,28.123383,0.090984,wv_ols_r2_F
6,IF,wv_ols_r2,"{""emphasize_diversity"": true}",0.205733,0.023293,0.248,215.401882,27.774810,0.113218,wv_ols_r2_T
7,KMeans,hard_voting,"{""spread_window"": 100, ""gaussian"": false}",0.426667,0.047321,0.524,3766.828459,1122.969759,0.110908,hard_voting_F
8,KMeans,hard_voting,"{""spread_window"": 100, ""gaussian"": true}",0.424133,0.048258,0.512,3767.096025,1122.969468,0.113780,hard_voting_T
9,KMeans,maximum_score,{},0.349733,0.034914,0.436,3767.332442,1122.962086,0.099829,maximum_score


# Note: 
when evaluating ensembles, we did not recompute the base learner anomaly score predictions. Instead, we simply extracted the relevant scores from the scores cached in Experiment 1.  The primary reason for this was computational cost.  This is a valid approach since none of the aggregation functions are dependent i.e. they do not have to be trained in sequence with the consitutent base learners.  However, this does mean that we did not record the _direct_ processing time per time series for any of the ensemble detectors.  Instead, we assumed that the base learners would make inference on the data in parallel, and took the score of the longest base learner plus the time for aggregation, as the processing time for the ensemble learner.

# CRUCIAL LIMITATION:

The computational time comparison between individual learners and ensembles may not be entirely fair. The ensemble processing times are not directly measured but estimated based on the longest base learner time plus aggregation time. This approach assumes parallel processing of base learners, which may not always be possible or practical.

In [9]:
import os
import pandas as pd
import re
import json

# Define the directory and file paths
input_dir = 'final_ensembles/results/'
output_file = 'final_experiments/demo.csv'

# Read existing data
existing_data = pd.read_csv(output_file)

# List to store new data
new_data = []

# Regular expression pattern to match desired files
pattern = r'ensemble_hom_(lof|if)_(.+)_rep([0-9])\.csv'

# Function to get method and params
def get_method_and_params(method_string):
    if method_string == 'simple_average':
        return 'simple_average', '{}'
    elif method_string == 'maximum_score':
        return 'maximum_score', '{}'
    elif method_string == 'wv_ols':
        return 'wv_ols', '{}'
    elif method_string.startswith('wv_ols_r2'):
        emphasize_diversity = method_string.endswith('_t')
        return 'wv_ols_r2', json.dumps({"emphasize_diversity": emphasize_diversity})
    elif method_string.startswith('hard_voting'):
        gaussian = method_string.endswith('_gaussian')
        return 'hard_voting', json.dumps({"spread_window": 100, "gaussian": gaussian})
    else:
        return method_string, '{}'  # Default case

# Iterate through files in the directory
for filename in os.listdir(input_dir):
    match = re.match(pattern, filename)
    if match and int(match.group(3)) < 10:  # Check if repetition is 0-9
        base_learner_type = match.group(1).upper()
        method_string = match.group(2)
        repetition = int(match.group(3))

        with open(os.path.join(input_dir, filename), 'r') as file:
            # Read the first line (header) and split it
            header = file.readline().strip().split(',')
            # Read the second line (data) and split it
            data = file.readline().strip().split(',')
            
            # Extract relevant information
            computational_time = float(data[1])
            ucr_score = float(data[2])
            
            # Get method and params
            ensemble_method, method_params = get_method_and_params(method_string)
            
            # Append to new_data list
            new_data.append({
                'base_learner_type': base_learner_type,
                'ensemble_method': ensemble_method,
                'method_params': method_params,
                'repetition': repetition,
                'ucr_score': ucr_score,
                'computational_time': computational_time
            })

# Convert new_data to DataFrame
new_df = pd.DataFrame(new_data)

# Append new data to existing data
combined_df = pd.concat([existing_data, new_df], ignore_index=True)

# Save the combined data back to the CSV file
combined_df.to_csv(output_file, index=False)

print(f"Data has been successfully appended to {output_file}")

Data has been successfully appended to final_experiments/demo.csv


/tmp/ipykernel_503469/337232195.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_data, new_df], ignore_index=True)


In [10]:
data = pd.read_csv(output_file)
sorted_data = data.sort_values(by=['base_learner_type', 'repetition', 'ensemble_method'])
sorted_data.to_csv('demo_2.csv', index=False)